#### Mounting Google Drive

In [54]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#### Setting up base paths

In [55]:
BASE_INPUT_PATH = "/content/drive/MyDrive/SP24-CSEN240-2/Project"
TRAIN_DATA_PATH = BASE_INPUT_PATH + "/Train"
VALIDATION_DATA_PATH = BASE_INPUT_PATH + "/Validation"
TEST_DATA_PATH = BASE_INPUT_PATH + "/Test"
GROUP_NUMBER = "9" # TODO
ADDITIONAL_FILES_PATH = BASE_INPUT_PATH + "/Temp/Group-%s" % GROUP_NUMBER
OUTPUT_PATH = BASE_INPUT_PATH +"/Prediction"

#### Import dependencies

In [56]:
!pip install mtcnn
!pip install keras-facenet

In [57]:
import cv2
import os
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas as pd
from google.colab.patches import cv2_imshow
from sklearn.preprocessing import LabelEncoder
from mtcnn.mtcnn import MTCNN
from keras_facenet import FaceNet
embedder = FaceNet()
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
import pickle
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

#### Preprocessing

In [58]:
# TODO
# TODO
detector=MTCNN()
def load_images_and_labels_with_face_detection(folder, label_mapping):
    images = []
    labels = []
    print_counter = 5
    for filename in sorted(os.listdir(folder)):
        file_number = filename.split('.')[0]
        img_path = os.path.join(folder, filename)
        img = cv2.imread(img_path)
        if img is not None:
            results = detector.detect_faces(img)
            if results:  # Check if any faces were detected
                x, y, w, h = results[0]['box']  # Get coordinates of the first detected face
                x, y = abs(x), abs(y)  # Ensure coordinates are positive
                face = img[y:y + h, x:x + w]  # Extract face region
                face = cv2.resize(face, (128, 128))  # Resize to consistent size
                images.append(face)
                # Determine label based on filename or folder structure (adjust as needed)
                label = label_mapping[file_number]
                labels.append(label)
                cv2.rectangle(img, (x, y), (x+w, y+h), (0, 255, 0), 2)
                cv2.waitKey(0)
            else:
                face=img
                face = cv2.resize(face, (128, 128))
                images.append(face)
                label = label_mapping[file_number]
                labels.append(label)

    return np.array(images), np.array(labels)



# Preprocessig the test data
# read images and labels, and detect the face boundary


In [59]:
class FACELOADING:
    def __init__(self, directory):
        self.directory = directory
        self.target_size = (160,160)
        self.X = []
        self.Y = []
        self.detector = MTCNN()

    def AB(self):
        self.X.extend(test_images)
        self.Y.extend(test_labels_encoded)
        return np.asarray(self.X), np.asarray(self.Y)

    def plot_images(self):
        plt.figure(figsize=(18,16))
        for num,image in enumerate(self.X):
            ncols = 3
            nrows = len(self.Y)//ncols + 1
            plt.subplot(nrows,ncols,num+1)
            plt.imshow(image)
            plt.axis('off')

Importing Pickle Files

In [60]:
import pickle

def load_from_pickle(file_path):
    with open(file_path, 'rb') as f:
        data = pickle.load(f)
    return data
train_images = load_from_pickle(ADDITIONAL_FILES_PATH +  '/train_images.pkl')
train_labels = load_from_pickle(ADDITIONAL_FILES_PATH +  '/train_labels.pkl')


filename_xy = os.path.join(ADDITIONAL_FILES_PATH, 'embedded_xy.pkl')
filename_uv = os.path.join(ADDITIONAL_FILES_PATH, 'embedded_uv.pkl')
with open(filename_xy, 'rb') as f:
    loaded_data = pickle.load(f)
    EMBEDDED_X, Y = loaded_data
with open(filename_uv, 'rb') as f:
    loaded_data = pickle.load(f)
    EMBEDDED_U, V = loaded_data



In [61]:
le = LabelEncoder()
train_labels = le.fit_transform(train_labels)
#val_labels = le.transform(val_labels)
#train_labels=sorted(train_labels)


Reading Test Images

In [62]:
test_labels_df = pd.read_csv(os.path.join(TEST_DATA_PATH, "labels.txt"), delimiter=" ", header=None, dtype = {0: str})
test_labels_df.columns = ["file_number", "person_name"]
# Create a dictionary to map file numbers to person names
test_label_mapping = test_labels_df.set_index('file_number')['person_name'].to_dict()

# Preprocessig the test data
# read images and labels, and detect the face boundary
test_images, test_labels = load_images_and_labels_with_face_detection(TEST_DATA_PATH, test_label_mapping)
# label encoding for test data
test_labels_encoded = le.transform(test_labels)
#test_labels_encoded=sorted(test_labels_encoded)

2/2 [==============================] - 0s 10ms/step


1/1 [==============================] - 0s 28ms/step


In [63]:
def get_embedding(face_img):
    face_img = face_img.astype('float32') # 3D(160x160x3)
    face_img = np.expand_dims(face_img, axis=0)
    # 4D (Nonex160x160x3)
    yhat= embedder.embeddings(face_img)
    return yhat[0] # 512D image (1x1x512)

faceevolving=FACELOADING(TEST_DATA_PATH)
A,B= faceevolving.AB()

In [64]:
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
Y = encoder.fit_transform(Y)
V= encoder.transform(V)
B= encoder.transform(B)

In [65]:
EMBEDDED_A = []

for img in A:
    EMBEDDED_A.append(get_embedding(img))

EMBEDDED_A = np.asarray(EMBEDDED_A)

1/1 [==============================] - 0s 51ms/step


#### Generate model performance reports

In [66]:

model = SVC(kernel='rbf', probability=True)
model.fit(EMBEDDED_X, Y)

SVC(probability=True)

In [67]:
ypreds_train = model.predict(EMBEDDED_X)
ypreds_val = model.predict(EMBEDDED_U)
ypreds_test = model.predict(EMBEDDED_A)

In [68]:
from sklearn.metrics import accuracy_score, classification_report

print("Training Accuracy:",accuracy_score(Y, ypreds_train))
print("Validation Accuracy:",accuracy_score(V,ypreds_val))
print("Test Accuracy:",accuracy_score(B,ypreds_test))

Training Accuracy: 1.0
Validation Accuracy: 0.9940119760479041
Test Accuracy: 1.0


#### Generate prediction file

In [69]:
from PIL import Image
Vprednames=le.inverse_transform(ypreds_test)
test_labels_df = pd.read_csv(os.path.join(TEST_DATA_PATH, "labels.txt"), delimiter=" ", header=None, dtype={0: str})
test_labels_df.columns = ["file_number", "person_name"]

predicted_mapping = {file_num: pred for file_num, pred in zip(test_labels_df['file_number'], Vprednames)}
    # print(predicted_mapping)
df = pd.DataFrame(list(predicted_mapping.items()), columns=['file_number', 'predicted'])
    # print("df", df)
df.to_csv(OUTPUT_PATH + '/' + 'Group-9-track-2.txt', sep=' ', header=False, index=False)


In [70]:
from sklearn.metrics import accuracy_score

with open(TEST_DATA_PATH+'/labels.txt', 'r') as f:
    y_test = [line.strip().split(' ')[1] for line in f.readlines()]

with open(OUTPUT_PATH+'/Group-9-track-2.txt', 'r') as f:
    y_pred = [line.strip().split(' ')[1] for line in f.readlines()]

score = accuracy_score(y_test, y_pred)
print('Accuracy score: %s'% score)

Accuracy score: 1.0
